# 1. Free Group
## 1.1. Generate a free group with generators {a,b,c,d}

In [7]:
F.<a,b,c,d> = FreeGroup(); F, F.an_element()
Id_F = F.one()

def degree(elm_of_F):
    pow_arr = [entry[1] for entry in elm_of_F.syllables()]
    return reduce(lambda x,y: x+y, pow_arr) if pow_arr != [] else 0

# degree(F.an_element()), degree(F.one())

In [8]:
a*b^2*c*d*d^-1
F([4]), F.gen(3), F.rank(), F.gens()
F.gens().index(d)
# 'a' == str(a)
# elm = a^1*b*b^-1*a^2*c; degree(elm)

3

## 1.2. Function which converts a string to a word of the free group

In [9]:
def to_gen(a_letter):
    a_letter_attr = dict(
        char = a_letter.lower(), 
        not_inv= (a_letter == a_letter.lower())
    )
    search_gen = [x for x in F.gens() if str(x) == a_letter_attr['char']]
    rtn = Id_F if search_gen == [] else search_gen[0]
    return rtn if a_letter_attr['not_inv'] else rtn.inverse()
    
def word(string):
    rtn = Id_F
    for gen in map(to_gen, list(string)):
        rtn = rtn * gen
    return rtn
def w(string):
    return word(string)    

In [10]:
myw = w('abAB');
myw, type(myw)

(a*b*a^-1*b^-1,
 <class 'sage.groups.free_group.FreeGroup_class_with_category.element_class'>)

# 2. Group ring of free group
## 2.1. Generate the group ring of the free group F

In [11]:
ZF = F.algebra(QQ); ZF.an_element(), ZF.one(), (ZF.one() + w('ab'))*w('c')

def r(elm_of_F_or_QQ):
    return elm_of_F_or_QQ*ZF.one()

def max_deg(elm_of_ZF):
    return max([degree(m[0]) for m in list(elm_of_ZF)])
    
def homo_parts(elm_of_ZF):
    homo_parts = [r(0)]*(max_deg(elm_of_ZF) + 1)
    for m in list(elm_of_ZF):
        deg = degree(m[0])
        homo_parts[deg] += r(m[1])*m[0] 
    return homo_parts

def modulo(elm_of_ZF, num):
    return reduce(lambda x,y: x+y, homo_parts(elm_of_ZF)[0:num])

In [12]:
elm = r(1)+r(2)*a-r(1)*b+r(1/2)*a*b + r(1)*b*a^2; elm
list(elm), max_deg(elm), homo_parts(elm), modulo(elm, 2) #,type(r(list(elm)[0][1]))

([(a*b, 1/2), (b, -1), (1, 1), (a, 2), (b*a^2, 1)],
 3,
 [1, 2*a - b, 1/2*a*b, b*a^2],
 1 + 2*a - b)

## 2.2. The abelianization of F and more

In [65]:
def abelianize(elm_of_F):
    rst = ZF.zero()
    for m in elm_of_F.syllables():
        rst += m[1]*r(m[0])
    return rst
def abel(elm):
    return abelianize(elm)

# 3. Magnus Expansions
## 3.1. Interface of expanders

In [16]:
class MagnusExpander:
    def __init__(self):
        self.moddeg = 4
    
    def expand(self, elm_in_F, modulo_degree = 4):
        self.moddeg = modulo_degree
        factors = []
        for x in elm_in_F.syllables():
            a_factor = self.expand_a_gen(x[0], (x[1] > 0) )
            factors.append(a_factor^abs(x[1]))
        return F.one() if factors == [] else modulo(reduce(lambda x, y: x*y, factors), self.moddeg)

    def expand_a_gen(self, gen_of_F, not_inverse):
        if not_inverse:
            return r(1) + r(gen_of_F) + r(0)
        else:
            return r(1) + (-1)*r(gen_of_F) + r(0)

In [17]:
th = MagnusExpander(); th.moddeg
myword = w('ab'); myword in F, myword.syllables()
result = th.expand(myword); print "{} --> {}".format(myword,result)

a*b --> 1 + a + b + a*b


## 3.2. The standard Magnus expansion

In [18]:
class Standard_Magexp(MagnusExpander):
    def __init__(self):
        MagnusExpander.__init__(self)
        
    def expand_a_gen(self, gen_of_F, not_inverse):
        if not_inverse:
            return r(1) + r(gen_of_F)
        else:
            a_factor = r(1)
            for i in range(1, self.moddeg):
                sign = -1^(i%2)            
                a_factor += (-1)^(i%2)*r(gen_of_F)^i
            return a_factor

In [19]:
mword = w('abAB'); mword, mword.syllables()
th_std = Standard_Magexp()
md = 5

result = th_std.expand(mword, md);
result, homo_parts(result)[2]

(1 + a*b - b*a - a*b*a - a*b^2 + b*a^2 + b*a*b + a*b*a^2 + (a*b)^2 + a*b^3 - b*a^3 - b*a^2*b - b*a*b^2,
 a*b - b*a)

## 3.3. The standard symplectic expansion

In [109]:
def bracket(elm1_of_F, elm2_of_F):
    return ZF.one()*elm1_of_F*elm2_of_F - ZF.one()*elm2_of_F*elm1_of_F

class Standard_sympexp(MagnusExpander):
    def __init__(self):
        MagnusExpander.__init__(self)
        
    def expand_a_gen(self, gen_of_F, not_inverse):
        idx = F.gens().index(gen_of_F) + 1   # a: 1, b: 2, c: 3, d: 4
        offset = 1 if is_odd(idx) else -1
        pair = (gen_of_F, F.gen(idx + offset - 1))
            
        a_factor = ZF.one()
        if not_inverse:
            a_factor += r(1)*pair[0] + r(1/2)*pair[0]^2 + r(1/2)*bracket(pair[0], pair[1])
        else:
            a_factor += -r(1)*pair[0] + r(1/2)*pair[0]^2 - r(1/2)*bracket(pair[0], pair[1])
        return a_factor
    
       
    def log_2(self, elm_of_F):
        hp = homo_parts(self.expand(elm_of_F))
        return hp[2] - r(1/2)*hp[1]^2

In [120]:
mword = w('caC'); mword, mword.syllables()
th_symp = Standard_sympexp();
md = 3

modulo(th_symp.expand(mword, md) - r(1), 2)
th_symp.expand(mword), th_symp.log_2(mword)

(1 + a + 1/2*a^2 + 1/2*a*b - a*c - 1/2*b*a + c*a - 1/2*a^2*c - 1/2*a*b*c + 1/2*a*c^2 - 1/2*a*c*d + 1/2*a*d*c + 1/2*b*a*c + 1/2*c*a^2 + 1/2*c*a*b - c*a*c - 1/2*c*b*a + 1/2*c^2*a - 1/2*c^2*d + 1/2*c*d*a - 1/2*d*c*a + 1/2*d*c^2,
 1/2*a*b - a*c - 1/2*b*a + c*a)

## 3.4. Coupling of two elements of ZF 

In [121]:
def intersection_form(gen1, gen2):
    indices = [F.gens().index(g) + 1 for g in [gen1,gen2]]
    # (1,2), (3,4) --> 1
    # (2,1), (4,3) --> -1
    if is_odd(indices[0]):
        inum = 1 if (indices[1] == indices[0] + 1) else 0
    else:
        inum = -1 if (indices[1] == indices[0] - 1) else 0
    return inum

def coupling_of_word_with_gen(elm_of_F, gen_of_F):
    rst = ZF.zero()
    head = elm_of_F.syllables()[0][0]
    rst += r(intersection_form(gen_of_F, head)) * head^(-1) * elm_of_F
    return rst

def coupling(elm1_of_ZF, elm2_of_ZF):
    mono_list1 = list(r(elm1_of_ZF))
    mono_list2 = list(r(elm2_of_ZF))
    rst = ZF.zero()
    for m1 in mono_list1:
        for m2 in mono_list2:
            if degree(m2[0]) == 1:
                rst += r(m1[1]*m2[1])*coupling_of_word_with_gen(m1[0], m2[0]) 
    return rst

In [184]:
mwd = w('aab')
elm = th_symp.log_2(mwd); abel_elm = abel(mwd);
ell = coupling(elm, abel_elm)
print "<{}, {}> = {}".format(elm, abel_elm, ell)

check = list(abel_elm)[0][0]; check
ell == abel_elm/dict(list(abel_elm))[check]*dict(list(ell))[check]

<3/2*a*b - 3/2*b*a, 2*a + b> = -3*a - 3/2*b


True

In [ ]:
tha = r(1) + r(1)*a + r(1/2)*a^2 + r(1/2)*bracket(a,b)
modulo(tha*( r(1) - r(1)*a + r(1/2)*a^2 - r(1/2)*bracket(a,b)),4)